In [973]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# House Prices

> Kaggle competition
> https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

# Data Preparation

In [974]:
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

## Null values

In [975]:
df_train_na = df_train.isna().sum()
df_train_na = df_train_na[df_train_na.notnull() & (df_train_na != 0)]

df_test_na = df_test.isna().sum()
df_test_na = df_test_na[df_test_na.notnull() & (df_test_na != 0)]

"df_train_na", df_train_na, "df_test_na", df_test_na

('df_train_na',
 LotFrontage      259
 Alley           1369
 MasVnrType         8
 MasVnrArea         8
 BsmtQual          37
 BsmtCond          37
 BsmtExposure      38
 BsmtFinType1      37
 BsmtFinType2      38
 Electrical         1
 FireplaceQu      690
 GarageType        81
 GarageYrBlt       81
 GarageFinish      81
 GarageQual        81
 GarageCond        81
 PoolQC          1453
 Fence           1179
 MiscFeature     1406
 dtype: int64,
 'df_test_na',
 MSZoning           4
 LotFrontage      227
 Alley           1352
 Utilities          2
 Exterior1st        1
 Exterior2nd        1
 MasVnrType        16
 MasVnrArea        15
 BsmtQual          44
 BsmtCond          45
 BsmtExposure      44
 BsmtFinType1      42
 BsmtFinSF1         1
 BsmtFinType2      42
 BsmtFinSF2         1
 BsmtUnfSF          1
 TotalBsmtSF        1
 BsmtFullBath       2
 BsmtHalfBath       2
 KitchenQual        1
 Functional         2
 FireplaceQu      730
 GarageType        76
 GarageYrBlt       78
 GarageF

### Garage

- **GarageType (cat):** Garage location (NA)
- **GarageYrBlt (num):** Year garage was built
- **GarageFinish (cat):** Interior finish of the garage (NA)
- **GarageCars (num):** Size of garage in car capacity
- **GarageArea (num):** Size of garage in square feet
- **GarageQual (cat):** Garage quality (NA)
- **GarageCond (cat):** Garage condition (NA)

In [976]:
garage_cols = [x for x in df_train.columns if "Garage" in x]
garage_cols

['GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond']

#### Garage categories

In [977]:
print("\033[1m    df_train    \033[0m",
      "\033[1m GarageCond\033[0m",df_train[df_train["GarageCond"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageQual\033[0m",df_train[df_train["GarageQual"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageFinish\033[0m",df_train[df_train["GarageFinish"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageType\033[0m",df_train[df_train["GarageType"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(), sep="\n")
print()
print("\033[1m    df_test    \033[0m",
      "\033[1m GarageCond\033[0m",df_test[df_test["GarageCond"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageQual\033[0m",df_test[df_test["GarageQual"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageFinish\033[0m",df_test[df_test["GarageFinish"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(),
      "\033[1m GarageType\033[0m",df_test[df_test["GarageType"].isna()][["GarageCars", "GarageArea", "GarageYrBlt"]].mean(), sep="\n")

    df_train    
 GarageCond
GarageCars     0.0
GarageArea     0.0
GarageYrBlt    NaN
dtype: float64
 GarageQual
GarageCars     0.0
GarageArea     0.0
GarageYrBlt    NaN
dtype: float64
 GarageFinish
GarageCars     0.0
GarageArea     0.0
GarageYrBlt    NaN
dtype: float64
 GarageType
GarageCars     0.0
GarageArea     0.0
GarageYrBlt    NaN
dtype: float64

    df_test    
 GarageCond
GarageCars     0.012987
GarageArea     4.675325
GarageYrBlt         NaN
dtype: float64
 GarageQual
GarageCars     0.012987
GarageArea     4.675325
GarageYrBlt         NaN
dtype: float64
 GarageFinish
GarageCars     0.012987
GarageArea     4.675325
GarageYrBlt         NaN
dtype: float64
 GarageType
GarageCars     0.0
GarageArea     0.0
GarageYrBlt    NaN
dtype: float64


There are no issue in the training set but some in the testing set.
For the testing set we will:
1. Keep GarageCond/GarageQual/GarageFinish/GarageType to NA if GarageCars = 0 & GarageArea = 0 and GarageYrBlt = NaN
2. If It is not the case we will try to estimate the missing values

In [978]:
df_test[(df_test["GarageCond"].isna() | 
         df_test["GarageQual"].isna() | 
         df_test["GarageFinish"].isna() | 
         df_test["GarageType"].isna()) & df_test["GarageCars"] != 0][garage_cols]

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
666,Detchd,NaN,NaN,1.0,360.0,NaN,NaN


In [979]:
df_test[(df_test["GarageCond"].isna() | 
         df_test["GarageQual"].isna() | 
         df_test["GarageFinish"].isna() | 
         df_test["GarageType"].isna()) & df_test["GarageArea"] != 0][garage_cols]

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
666,Detchd,NaN,NaN,1.0,360.0,NaN,NaN


It's seem not worth it to try to estimate this line values.

In [980]:
# Fill NaN categories with None
# That is to say No garage
for df in [df_train, df_test]:
    df["GarageCond"] = df["GarageCond"].fillna("None")
    df["GarageQual"] = df["GarageQual"].fillna("None")
    df["GarageFinish"] = df["GarageFinish"].fillna("None")
    df["GarageType"] = df["GarageType"].fillna("None")

#### Garage nums

In [981]:
print("df_train", df_train[((df_train["GarageCond"] == "None") | 
          (df_train["GarageQual"] == "None") | 
          (df_train["GarageFinish"] == "None") | 
          (df_train["GarageType"] == "None")) & df_train["GarageYrBlt"].isna()].shape[0], df_train["GarageYrBlt"].isna().sum(),
      "df_test", df_test[((df_test["GarageCond"] == "None") | 
         (df_test["GarageQual"] == "None") | 
         (df_test["GarageFinish"] == "None") | 
         (df_test["GarageType"] == "None")) & df_test["GarageYrBlt"].isna()].shape[0], df_test["GarageYrBlt"].isna().sum(), sep="\n")

df_train
81
81
df_test
78
78


In [982]:
# There are only NaN values for absence of garage 
# Set value to specific constant
df_test["GarageYrBlt"] = df_test["GarageYrBlt"].fillna(0)
df_train["GarageYrBlt"] = df_train["GarageYrBlt"].fillna(0)

In [983]:
df_test[df_test["GarageCars"].isna()][garage_cols]

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
1116,Detchd,0.0,None,NaN,NaN,None,None


In [984]:
# It's seem like there is no garage for this line after all
df_test["GarageCars"] = df_test["GarageCars"].fillna(0)
df_test["GarageArea"] = df_test["GarageArea"].fillna(0)

### Basement

- **BsmtCond (cat):** Evaluates the general condition of the basement (NA)
- **BsmtQual (cat):** Evaluates the height of the basement (NA)
- **BsmtExposure (cat):** Refers to walkout or garden level walls (NA)
- **BsmtFinType1 (cat):** Rating of basement finished area (NA)
- **BsmtFinType2 (cat):** Rating of basement finished area (if multiple types) (NA)
- **BsmtFinSF1 (num):** Type 1 finished square feet
- **BsmtFinSF2 (num):** Type 2 finished square feet
- **BsmtUnfSF (num):** Unfinished square feet of basement area
- **TotalBsmtSF (num):** Total square feet of basement area
- **BsmtFullBath (num):** Basement full bathrooms
- **BsmtHalfBath (num):** Basement half bathrooms

In [985]:
bsmt_cols = [x for x in df_train.columns if "Bsmt" in x]
bsmt_cat_cols = ["BsmtCond", "BsmtQual", "BsmtExposure", "BsmtFinType1", "BsmtFinType2"]
bsmt_num_cols = list(set(bsmt_cols)-set(bsmt_cat_cols))
bsmt_cols

['BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath']

#### Basement nums

In [986]:
"df_train", df_train[bsmt_num_cols].isna().sum(), "df_test", df_test[bsmt_num_cols].isna().sum(),

('df_train',
 BsmtFinSF2      0
 BsmtFullBath    0
 BsmtHalfBath    0
 BsmtFinSF1      0
 BsmtUnfSF       0
 TotalBsmtSF     0
 dtype: int64,
 'df_test',
 BsmtFinSF2      1
 BsmtFullBath    2
 BsmtHalfBath    2
 BsmtFinSF1      1
 BsmtUnfSF       1
 TotalBsmtSF     1
 dtype: int64)

In [987]:
df_test[df_test["BsmtFinSF1"].isna() |
        df_test["BsmtFinSF2"].isna() | 
        df_test["BsmtHalfBath"].isna() | 
        df_test["BsmtFullBath"].isna() | 
        df_test["BsmtUnfSF"].isna() | 
        df_test["TotalBsmtSF"].isna()][bsmt_cols]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath
660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
728,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN


In [988]:
# We can just put values to 0
for c in bsmt_num_cols:
    df_test[c] = df_test[c].fillna(0)

#### Basement categories

In [989]:
"df_train", df_train[bsmt_cat_cols].isna().sum(), "df_test", df_test[bsmt_cat_cols].isna().sum(),

('df_train',
 BsmtCond        37
 BsmtQual        37
 BsmtExposure    38
 BsmtFinType1    37
 BsmtFinType2    38
 dtype: int64,
 'df_test',
 BsmtCond        45
 BsmtQual        44
 BsmtExposure    44
 BsmtFinType1    42
 BsmtFinType2    42
 dtype: int64)

In [990]:
# If the TotalBsmtSF is 0, NaN means no basement with hight chance
for df in [df_train, df_test]:
    for cat in bsmt_cat_cols:
        df.loc[df["TotalBsmtSF"] == 0, cat] = df.loc[df["TotalBsmtSF"] == 0, cat].fillna("None")

In [991]:
# Specificaly for BsmtFinType2 we will use BsmtFinSF2
for df in [df_train, df_test]:
    df.loc[df["BsmtFinSF2"] == 0 , "BsmtFinType2"] = df.loc[df["BsmtFinSF2"] == 0, "BsmtFinType2"].fillna("None")

In [992]:
# For the rest of NaN values we will create the Ukn value
for df in [df_train, df_test]:
    for cat in bsmt_cat_cols:
        df[cat] = df[cat].fillna("None")

### Other columns

In [993]:
# Ultra basic nan values to mean
def to_mean(df):
    # Select all numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    # NaN to mean
    for c in numeric_columns:
        df[c] = df[c].fillna(df[c].mean())
    return df

In [994]:
df_train = to_mean(df_train)
df_test = to_mean(df_test)

## Categorization


In [995]:
# Categorization function
def to_categories(df, exception=["Id"]):
    # Select all non-numeric columns
    non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
    # Filter exception
    non_numeric_columns = list(set(non_numeric_columns) - set(exception))
    # Convert the non-numeric columns to categorical
    for c in non_numeric_columns:
        df[c] = df[c].astype('category').cat.codes + 1
    return df

In [996]:
# Specific categorization preparation
cat_cols = list(df_train.select_dtypes(exclude=[np.number]).columns)

for c in cat_cols:
    catcode = df_train[[c, "SalePrice"]].groupby(c).mean().sort_values("SalePrice").reset_index()
    for df in [df_train, df_test]:
        for index, row in catcode.iterrows():
            df.loc[df[c] == row[c], c] = index

In [997]:
df_train = to_categories(df_train)
df_test = to_categories(df_test)

## Normalization

In [998]:
def max_abs(df, exception=["Id", "SalePrice"]):
    for column in list(set(df.columns) - set(exception)):
        df[column] = df[column]  / df[column].abs().max()

def min_max(df, exception=["Id", "SalePrice"]):
    for column in list(set(df.columns) - set(exception)):
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())

def z_score_normalization(df, exception=["Id", "SalePrice"]):
    for column in list(set(df.columns) - set(exception)):
        df[column] = (df[column] - df[column].mean()) / df[column].std()

In [999]:
z_score_normalization(df_train)
z_score_normalization(df_test)

In [1000]:
df_train.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0.073350,0.358921,-0.229293,-0.207071,0.064216,-0.242942,-0.656815,-0.111129,0.026171,...,-0.068668,-0.063283,-0.469706,-0.192408,-0.087658,-1.598563,0.138730,-0.238924,0.189904,208500
1,2,-0.872264,0.358921,0.451781,-0.091855,0.064216,-0.242942,-0.656815,-0.111129,0.026171,...,-0.068668,-0.063283,-0.469706,-0.192408,-0.087658,-0.488943,-0.614228,-0.238924,0.189904,181500
2,3,0.073350,0.358921,-0.093078,0.073455,0.064216,-0.242942,0.872610,-0.111129,0.026171,...,-0.068668,-0.063283,-0.469706,-0.192408,-0.087658,0.990552,0.138730,-0.238924,0.189904,223500


In [1001]:
X_train = df_train[list(set(df_train.columns) - set(["Id", "SalePrice"]))]
y_train = df_train["SalePrice"]

X_sub = df_test[list(set(df_test.columns) - set(["Id"]))]
y_sub = df_test["Id"]

# Model

In [1002]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor

import xgboost as xgb

In [1003]:
model_r = Ridge(alpha=300, fit_intercept=True, max_iter=20000, solver='saga').fit(X_train, y_train)
print("model_r:", cross_val_score(model_r, X_train, y_train).mean())

model_r: 0.8230951105766076


In [1004]:
model_l = Lasso(alpha=1000, max_iter=10000, tol=0.001).fit(X_train, y_train)
print("model_l:", cross_val_score(model_l, X_train, y_train).mean())

model_l: 0.8197330217190176


In [1005]:
model_en = ElasticNet(alpha=3, l1_ratio=0.9, max_iter=1000, fit_intercept=True).fit(X_train, y_train)
print("model_en:", cross_val_score(model_en, X_train, y_train).mean())

model_en: 0.8230127810387543


In [1006]:
model_dtr = DecisionTreeRegressor(max_depth=10, min_samples_split=30, min_samples_leaf=2, max_features='auto').fit(X_train, y_train)
print("model_dtr:", cross_val_score(model_dtr, X_train, y_train).mean())

model_dtr: 0.752583501657268


In [1007]:
model_xgb = xgb_model = xgb.XGBRegressor(
    max_depth=5,
    learning_rate=0.1,
    n_estimators=500,
    subsample=0.75,
    colsample_bytree=0.5,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror'
).fit(X_train, y_train)
print("model_xgb:", cross_val_score(model_xgb, X_train, y_train).mean())

model_xgb: 0.9013998190254273


In [1008]:
estimators = [('model_r', model_r), ('model_l', model_l), ('model_en', model_en), ('model_dtr', model_dtr), ('model_xgb', model_xgb)]
ensemble = VotingRegressor(estimators=estimators, weights=[0.1, 0.1, 0.1, 0.1, 0.6]).fit(X_train, y_train)
print("ensemble:", cross_val_score(ensemble, X_train, y_train).mean())

ensemble: 0.893500652777625


No null cross_val_score:
- model_r: 0.8224796896298135
- model_l: 0.8148001943085397
- model_en: 0.822370042604566
- model_dtr: 0.7721313160197425
- model_xgb: 0.8890202554272479
- ensemble: 0.8847559550521403

In [1009]:
prediction = ensemble.predict(X_sub)
prediction

array([122783.99352953, 167250.28557105, 175965.90422928, ...,
       166502.08073634, 114951.20841152, 207179.97949426])

In [1010]:
df_pred = pd.DataFrame({"Id":y_sub, "SalePrice":prediction})
df_pred.head(3)

,Id,SalePrice
0,1461,122783.993530
1,1462,167250.285571
2,1463,175965.904229


In [1011]:
df_pred.to_csv("/kaggle/working/submission.csv",index=False)